In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# We need to install a wide variety of libraries. For this we will install pandas, numpy, seaborn and matplotlib libraries.

import numpy as np
import pandas as pd
import seaborn as sns
import string
import re
sns.set()

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings("ignore")

# Graphics in retina format are more sharp and legible
%config InlineBackend.figure_format = 'retina'

In [3]:
df = pd.read_csv('/kaggle/input/project-check/reviews.csv')
df.head()

In [4]:
X = df['titletext']
y = df['label']
y.value_counts()

## 1- Loading the data.

In [19]:
# split into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,  random_state=42, stratify=y)
len(X_train), len(y_train), len(X_test), len(y_test)

In [20]:
X_train

In [21]:
y_train

In [22]:
y_train.value_counts()

**CountVectorizer**

In [51]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

**Tf-idf**

`TF-IDF` **Term Frequency–Inverse Document Frequency**, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.
The tf–idf value increases proportionally to the number of times a word appears in the document and is offset by the number of documents in the corpus that contain the word, which helps to adjust for the fact that some words appear more frequently in general.

In [52]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
X_train_tfidf = tfidf.fit_transform(X_train_cv)

## 5- Modeling

### multinomial Naive Bayes
We will use `multinomial Naive Bayes`


The multinomial Naive Bayes classifier is suitable for classification with discrete features

In [53]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train_cv, y_train)

In [55]:
# predict classes
y_pred = nb.predict(X_test_cv)
y_pred_proba = nb.predict_proba(X_test_cv)[:, 1]

In [57]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score

In [58]:
p, r, f, _ = precision_recall_fscore_support(y_test, y_pred,
                                                 average='macro')
print(f'precision is  -> {round(p,2)}')
print(f'recall is -> {round(r,2)}')
print(f'f1_score is -> {round(f,2)}')

In [59]:
# calculate the accuracy 
acc = accuracy_score(y_test, y_pred)
print(f'Accuracy is -> {round(acc,2)}')

In [60]:
cm = confusion_matrix(y_test, y_pred)
print('The confusion_matrix is: \n', cm);

In [61]:
from mlxtend.plotting import plot_confusion_matrix

fig, ax = plot_confusion_matrix(conf_mat= cm, colorbar=True)
plt.show()

In [ ]:
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f'The roc_auc Score is -> {round(roc_auc,2)}')

### Make it by pipline

**`MultinomialNB with TF-idf`**

In [62]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

pipe = Pipeline([('bow', CountVectorizer()), 
                 ('tfidf', TfidfTransformer()),  
                 ('model', MultinomialNB())])

pipe.fit(X_train, y_train)

In [64]:
y_pred = pipe.predict(X_test)

In [65]:
accuracy_score(y_test, y_pred)

In [66]:
confusion_matrix(y_test, y_pred)

In [67]:
print(classification_report(y_test, y_pred))

### XGboost

**`XGboost with TF-idf`**

In [41]:
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, classification_report

pipe = Pipeline([
                ('bow', CountVectorizer()), 
                ('tfidf', TfidfTransformer()),  
                ('model', xgb.XGBClassifier(
                    learning_rate=0.1,
                    max_depth=7,
                    n_estimators=80,
                    use_label_encoder=False,
                    eval_metric='auc'))
                ])


In [42]:
# Fit the pipeline with the data
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
# y_pred_train = pipe.predict(X_train)

# print('Train: {}'.format(accuracy_score(y_train, y_pred_train)))
print(f'Accuracy Score For Test Data: -> {round(accuracy_score(y_test, y_pred),2)}')
print(f'Confusion Matrix Score For Test Data:\n {confusion_matrix(y_test, y_pred)}')

In [43]:
print(classification_report(y_test, y_pred))

In [44]:
# calculate the accuracy 
acc = accuracy_score(y_test, y_pred)
print(f'Accuracy is -> {round(acc,2)}')